In [1]:
import pandas as pd
from scipy import stats
import numpy as np
import os

In [2]:
def get_dataframe_for_pvalue (path) :
  df_final = pd.DataFrame()
  for file in path :
      df = pd.read_excel(file)
      num = 0
      drop_list = []
      while num in df.index :
          value = df.iat[num,1]
          try :
              value = float(value)
          except :
              drop_list.append(num)
          num +=1
      df1 = df.drop(drop_list)
      if path.index(file) == 0 :
          df_final = pd.concat([df_final , df1.iloc[:,[0]]] , axis = 1)
      df_final = pd.concat([df_final,df1.iloc[:,[1]]],axis = 1)
  return (df_final)

In [3]:
patients_path = r'/content/drive/MyDrive/esophageal_patients_data (1).xlsx'
df_patients = pd.read_excel(patients_path,index_col=0)
df_fillna_0 = df_patients.fillna(0)
index1 = df_patients.loc[df_patients[ 'local recurrence:\n1) in field   \n2) near  \n3) out field    \n4) in and out(ex: LAP)']>=1].index
index0 = df_fillna_0.loc[df_fillna_0[ 'local recurrence:\n1) in field   \n2) near  \n3) out field    \n4) in and out(ex: LAP)'] == 0].index

In [4]:
radfile_dir = r'/content/drive/Shareddrives/食道癌/rad_pet_0-64'
extracter = os.listdir(radfile_dir)
for dir in extracter :
  radfile_fullpath_index1 = []
  radfile_fullpath_index0 = []
  dir_path = os.path.join(radfile_dir , dir)
  file_list = os.listdir(dir_path)
  for file  in file_list :
    fullpath = os.path.join(dir_path , file)
    if os.path.isfile(fullpath) :
      for i in index1 :
        if str(i) in file :
          radfile_fullpath_index1.append(fullpath)
      for j in index0 :
        if str(j) in file :
          radfile_fullpath_index0.append(fullpath)
  df_final0 = get_dataframe_for_pvalue(radfile_fullpath_index0)
  df_final1 = get_dataframe_for_pvalue(radfile_fullpath_index1)
  t_statistic = []
  p_value = []
  for i in range(len(df_final1.index)) :
    list_parameters1 = []
    max_num = len(df_final1.columns) - 1
    num = 2
    while num <= max_num :
      list_parameters1.append(df_final1.iat[i,num])
      num +=1
    list_parameters0 = []
    max_num = len(df_final0.columns) - 1
    num = 2
    while num <= max_num :
      list_parameters0.append(df_final0.iat[i,num])
      num +=1
    list_parameters1 = np.array(list_parameters1).astype(np.float64)
    list_parameters0 = np.array(list_parameters0).astype(np.float64)
    t , p = stats.ttest_ind(list_parameters1, list_parameters0)
    t_statistic.append(t)
    p_value.append(p)
  feature_list = []
  for i in range(len(df_final0.index)) :
    feature_list.append(df_final0.iat[i,0])
  df_pvalue = pd.DataFrame({'t_statistic' : t_statistic , 'p_value' : p_value}
               ,index = feature_list)
  save_dir = r'/content/drive/Shareddrives/食道癌/pet_0-64_pvalue_local_re/'
  savepath = save_dir + dir +'_pvalue結果.xlsx'
  df_pvalue.to_excel(savepath)

In [ ]:
print(df_pvalue)

                                     t_statistic   p_value
diagnostics_Image-original_Mean        -1.606733  0.110244
diagnostics_Image-original_Minimum           NaN       NaN
diagnostics_Image-original_Maximum           NaN       NaN
diagnostics_Mask-original_VoxelNum      0.895483  0.371983
diagnostics_Mask-original_VolumeNum    -1.051957  0.294534
original_glcm_JointEntropy                   NaN       NaN


In [ ]:
print(df_final1.index)

Int64Index([11, 12, 13, 18, 19, 22], dtype='int64')


In [ ]:
print(df_final0.index)

Int64Index([11, 12, 13, 18, 19, 22], dtype='int64')


In [ ]:
print(len(list_parameters0),len(list_parameters1))

63 87
